In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, classification_report, roc_auc_score, plot_roc_curve
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [6]:
data = pd.read_csv(r'C:\Users\ASUS\Downloads\Detection of parkinsons disease using voice\New_train_data.csv')
datadrop=data.drop(['Subject_ID'], axis=1)

In [7]:
X = datadrop.iloc[:,0:26]  #independent columns
y = datadrop.iloc[:,-1]    #target column i.e price range

In [8]:
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features


                                  Specs        Score
18                        Maximum Pitch  1269.810767
16                   Standard Deviation   743.564627
20                    Number of Periods   485.662166
19                     Number of Pulses   358.876408
25               Degree of Voice Breaks   307.212555
23  Fraction of Locally Unvoiced Frames   245.952957
15                           Mean Pitch   205.726319
14                         Median Pitch   141.583040
9                        Shimmer(apq11)    65.918170
17                        Minimun Pitch    59.740351


In [13]:
datadrop1=data.drop(['Subject_ID','Jitter(local)','Jitter(local, absolute)','Jitter(rap)','Jitter(ppq5)','Jitter(ddp)','Shimmer(local)','Shimmer(local, dB)','Shimmer(apq3)','Shimmer(apq5)','Shimmer(dda)','AC','NTH','HTN','Number of Voice Breaks','UPDRS','Mean Period','Standard Deviation of Period'], axis=1)

In [14]:
datadrop1.dtypes

Shimmer(apq11)                         float64
Median Pitch                           float64
Mean Pitch                             float64
Standard Deviation                     float64
Minimun Pitch                          float64
Maximum Pitch                          float64
Number of Pulses                         int64
Number of Periods                        int64
Fraction of Locally Unvoiced Frames    float64
Degree of Voice Breaks                 float64
Class_Information                        int64
dtype: object

In [17]:
features1=datadrop1.loc[:,datadrop1.columns!='Class_Information'].values[:,1:]
labels=datadrop1.loc[:,'Class_Information'].values

In [18]:
print(labels[labels==1].shape[0], labels[labels==0].shape[0])

520 520


In [23]:
scaler1=MinMaxScaler((-1,1))
x=scaler1.fit_transform(features1)
y=labels

In [33]:
x_train,y_train=x, y

In [34]:
model = XGBClassifier(eval_metric='mlogloss')

In [35]:
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [36]:
x_test=pd.read_csv(r'C:\Users\ASUS\Downloads\Detection of parkinsons disease using voice\New_test_data.csv')

In [37]:

datadrop2=x_test.drop(['Subject_ID','Jitter(local)','Jitter(local, absolute)','Jitter(rap)','Jitter(ppq5)','Jitter(ddp)','Shimmer(local)','Shimmer(local, dB)','Shimmer(apq3)','Shimmer(apq5)','Shimmer(dda)','AC','NTH','HTN','Number of Voice Breaks','Mean Period','Standard Deviation of Period'], axis=1)


In [38]:
features1=datadrop2.loc[:,datadrop2.columns!='Class_Information'].values[:,1:]
labels=datadrop2.loc[:,'Class_Information'].values

In [39]:
print(labels[labels==1].shape[0], labels[labels==0].shape[0])

168 0


In [40]:
scaler=MinMaxScaler((-1,1))
x1=scaler.fit_transform(features1)
y1=labels

In [41]:
# Finally, generate y_pred (predicted values for x_test)

y_pred=model.predict(x1)

In [42]:
# Calculate the accuracy for the model. Print it out.

print(accuracy_score(y1, y_pred))

# 92.3076923076923 -with MinMaxscalar

0.48214285714285715
